# Some Models
> To avoid copy-pasta #2

In [ ]:
# default_exp models

In [ ]:
# hide
import blackhc.project.script
from nbdev.showdoc import *

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


In [ ]:
# exports
from dataclasses import dataclass
from typing import Optional

import torch
import torch.nn
import torch.optim
from torch import nn as nn
from torch.nn import functional as F, Module
from torch.utils.data import DataLoader, Dataset

from batchbald_redux.active_learning import RandomFixedLengthSampler
from batchbald_redux.black_box_model_training import train
from batchbald_redux.consistent_mc_dropout import (
    GradEmbeddingType,
    BayesianModule,
    ConsistentMCDropout,
    ConsistentMCDropout2d,
    freeze_encoder_context
)

from batchbald_redux.model_optimizer_factory import ModelOptimizer, ModelOptimizerFactory

In [ ]:
# exports
from batchbald_redux.trained_model import ModelTrainer, TrainedModel, TrainedBayesianModel


class BayesianMNISTCNN(BayesianModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_drop = ConsistentMCDropout2d()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = ConsistentMCDropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc1_drop = ConsistentMCDropout()
        self.fc2 = nn.Linear(128, num_classes)

    def mc_forward_impl(self, input: torch.Tensor, freeze_encoder: bool):
        with freeze_encoder_context(freeze_encoder):
            input = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(input)), 2))
            input = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(input)), 2))
            input = input.view(-1, 1024)
            input = F.relu(self.fc1_drop(self.fc1(input)))

        embedding = input
        input = self.fc2(input)
        input = F.log_softmax(input, dim=1)

        return input, embedding

In [ ]:
BayesianMNISTCNN()

BayesianMNISTCNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv1_drop): ConsistentMCDropout2d(p=0.5)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): ConsistentMCDropout2d(p=0.5)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc1_drop): ConsistentMCDropout(p=0.5)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# exports


class BayesianMNISTCNN_EBM(BayesianModule):
    """Without Softmax."""

    def __init__(self, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_drop = ConsistentMCDropout2d()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = ConsistentMCDropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc1_drop = ConsistentMCDropout()
        self.fc2 = nn.Linear(128, num_classes)

    def mc_forward_impl(self, input: torch.Tensor, freeze_encoder: bool):
        with freeze_encoder_context(freeze_encoder):
            input = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(input)), 2))
            input = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(input)), 2))
            input = input.view(-1, 1024)
            input = F.relu(self.fc1_drop(self.fc1(input)))

        embedding = input
        input = self.fc2(input)

        return input, embedding

In [ ]:
BayesianMNISTCNN_EBM()

BayesianMNISTCNN_EBM(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv1_drop): ConsistentMCDropout2d(p=0.5)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): ConsistentMCDropout2d(p=0.5)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc1_drop): ConsistentMCDropout(p=0.5)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# exports


class MnistOptimizerFactory(ModelOptimizerFactory):
    def create_model_optimizer(self) -> ModelOptimizer:
        model = BayesianMNISTCNN()
        optimizer = torch.optim.Adam(model.parameters(), weight_decay=5e-4)
        return ModelOptimizer(model=model, optimizer=optimizer)


@dataclass
class MnistModelTrainer(ModelTrainer):
    device: str

    num_training_samples: int = 1
    num_validation_samples: int = 20
    num_patience_epochs: int = 20
    max_training_epochs: int = 120

    min_samples_per_epoch: int = 1024
    num_training_batch_size: int = 64
    num_evaluation_batch_size: int = 128

    @staticmethod
    def create_model_optimizer() -> ModelOptimizer:
        model = BayesianMNISTCNN()
        optimizer = torch.optim.Adam(model.parameters(), weight_decay=5e-4)
        return ModelOptimizer(model=model, optimizer=optimizer)

    def get_train_dataloader(self, dataset: Dataset):
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=self.num_training_batch_size,
            sampler=RandomFixedLengthSampler(dataset, self.min_samples_per_epoch),
            drop_last=True,
        )
        return train_loader

    def get_evaluation_dataloader(self, dataset: Dataset):
        evaluation_loader = torch.utils.data.DataLoader(
            dataset, batch_size=self.num_evaluation_batch_size, drop_last=False, shuffle=False
        )
        return evaluation_loader

    def get_trained(self, *, train_loader: DataLoader, train_augmentations: Optional[Module],
                    validation_loader: DataLoader, log, wandb_key_path:str, loss=None, validation_loss=None) -> TrainedModel:
        model_optimizer = self.create_model_optimizer()

        if loss is None:
            loss = torch.nn.NLLLoss()
        if validation_loss is None:
            validation_loss = torch.nn.NLLLoss()

        train(
            model=model_optimizer.model,
            optimizer=model_optimizer.optimizer,
            training_samples=self.num_training_samples,
            validation_samples=self.num_validation_samples,
            train_loader=train_loader,
            train_augmentations=train_augmentations,
            validation_loader=validation_loader,
            patience=self.num_patience_epochs,
            max_epochs=self.max_training_epochs,
            loss=loss,
            validation_loss=validation_loss,
            device=self.device,
            training_log=log,
            wandb_key_path=wandb_key_path,
        )

        return TrainedBayesianModel(model_optimizer.model)

In [ ]:
# slow

from batchbald_redux import dataset_challenges

fast_mnist_train, fast_mnist_test = dataset_challenges.create_MNIST_dataset("cuda")

model_trainer = MnistModelTrainer("cuda", max_training_epochs=1)

train_loader = model_trainer.get_train_dataloader(fast_mnist_train)

test_loader = model_trainer.get_evaluation_dataloader(fast_mnist_test)

log = {}
trained_model = model_trainer.get_trained(train_loader=train_loader, train_augmentations=None, validation_loader=test_loader, log=log)

100%|##########| 1/1 [00:00<?, ?it/s]

[1/937]   0%|           [00:00<?]

Engine run is terminating due to exception: .


KeyboardInterrupt: 

In [ ]:
# slow

subset_mnist = fast_mnist_train * 0.2
print(subset_mnist)

train_loader = model_trainer.get_train_dataloader(subset_mnist)

test_loader = model_trainer.get_evaluation_dataloader(fast_mnist_test)

log = {}
trained_model = model_trainer.get_trained(train_loader=train_loader, train_augmentations=None, validation_loader=test_loader, log=log)

('FastMNIST (Train)')~x0.2


100%|##########| 1/1 [00:00<?, ?it/s]

[1/187]   1%|           [00:00<?]

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.9388, 'crossentropy': 0.36818689460754395}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.9388)
RestoringEarlyStopping: Restoring optimizer.


In [ ]:
# slow

trained_model.model.get_embeddings(num_samples=0, loader=test_loader, device="cuda", storage_device="cpu").shape


get_predictions_labels: |          | 0/0 [00:00<?, ?it/s]

torch.Size([10000, 1, 1024])

In [ ]:
# slow

trained_model.model.get_grad_embeddings(num_samples=0, loader=test_loader, loss=torch.nn.functional.nll_loss, grad_embedding_type=GradEmbeddingType.BIAS_LINEAR, model_labels=False, device="cuda", storage_device="cpu").shape

get_predictions_labels: |          | 0/0 [00:00<?, ?it/s]

torch.Size([10000, 1, 10250])

In [ ]:
trained_model.model.get_grad_embeddings(num_samples=0, loader=test_loader, loss=torch.nn.functional.nll_loss, grad_embedding_type=GradEmbeddingType.BIAS_LINEAR, model_labels=True, device="cuda", storage_device="cpu").shape

get_predictions_labels: |          | 0/0 [00:00<?, ?it/s]

torch.Size([10000, 1, 10250])